<a href="https://colab.research.google.com/github/financieras/gym/blob/main/frozen_lake/map_frozen_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapas para el juego Frozen Lake
Este juego lo resolveremos con aprendizaje automático usando el algoritmo Q-Learning y crearemos la Q-table.

## Determinar si un mapa es válido

In [1]:
def validar_mapa(mapa):
    """
    Valida un mapa de Frozen Lake según las especificaciones actualizadas.

    Args:
        mapa: Lista de strings que representan el mapa (nxn con n ≥ 2).
              S: Punto inicial, ·: Superficie helada, H: Hoyo, G: Meta.

    Returns:
        tuple: (bool, str) donde:
               - bool: True si el mapa es válido, False si no.
               - str: Mensaje descriptivo del resultado o error.
    """
    # Validación básica de tipo y estructura
    if not isinstance(mapa, list) or not mapa:
        return False, "El mapa debe ser una lista no vacía"

    n = len(mapa)
    if n < 2:
        return False, "El mapa debe tener al menos 2 filas"

    # Verificación de estructura cuadrada (nxn)
    if not all(isinstance(fila, str) and len(fila) == n for fila in mapa):
        return False, f"El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud {n}"

    # Conteo de caracteres especiales y validación
    caracteres_validos = {'S', '·', 'H', 'G'}
    caracteres_presentes = set()
    contador_S = 0
    contador_G = 0

    for fila in mapa:
        for c in fila:
            caracteres_presentes.add(c)
            if c == 'S':
                contador_S += 1
            elif c == 'G':
                contador_G += 1

    # Validación de caracteres
    caracteres_invalidos = caracteres_presentes - caracteres_validos
    if caracteres_invalidos:
        return False, f"Caracteres inválidos encontrados: {', '.join(sorted(caracteres_invalidos))}"

    # Validación de S y G
    if contador_S != 1:
        return False, f"Debe haber exactamente un 'S' (inicio). Encontrados: {contador_S}"

    if contador_G != 1:
        return False, f"Debe haber exactamente un 'G' (meta). Encontrados: {contador_G}"

    return True, f"Mapa {n}x{n} válido"

In [2]:
# Mapa válido
mapa_valido = [
    "S···",
    "·H·H",
    "··H·",
    "H··G"
]
print(validar_mapa(mapa_valido))  # (True, "Mapa 4x4 válido")

# Mapa con caracteres inválidos
mapa_invalido = [
    "S·X·",
    "·H·H",
    "··H·",
    "H··G"
]
print(validar_mapa(mapa_invalido))  # (False, "Caracteres inválidos encontrados: X")

# Caso inválido (fila 2 más corta)
mapa2 = [
    "S···",
    "·H·H",
    "···",  # ¡Falta un carácter!
    "H··G"
]

print(validar_mapa(mapa2))  # (False, 'El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud 4')

(True, 'Mapa 4x4 válido')
(False, 'Caracteres inválidos encontrados: X')
(False, 'El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud 4')


## Calcular cuántos caminos minimos existen y el número de pasos necesarios

In [3]:
from collections import deque

def contar_caminos_minimos(mapa):
    """
    Encuentra el número de caminos mínimos y su longitud desde S hasta G.

    Args:
        mapa: Lista de strings que representan el mapa (debe ser válido).

    Returns:
        tuple: (num_caminos, pasos) donde:
               - num_caminos: Número de caminos mínimos (0 si no hay camino)
               - pasos: Número de pasos del camino mínimo (-1 si no hay camino)
    """
    # Validación inicial del mapa (omitiendo mensajes para brevedad)
    valido, _ = validar_mapa(mapa)
    if not valido:
        return 0, -1

    # Encontrar posiciones de S y G
    S_pos = next((i, j) for i, row in enumerate(mapa)
                   for j, c in enumerate(row) if c == 'S')
    G_pos = next((i, j) for i, row in enumerate(mapa)
                   for j, c in enumerate(row) if c == 'G')

    # Inicialización
    filas, cols = len(mapa), len(mapa[0])
    direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    distancia = [[-1] * cols for _ in range(filas)]
    contador = [[0] * cols for _ in range(filas)]

    # BFS
    q = deque([S_pos])
    distancia[S_pos[0]][S_pos[1]] = 0
    contador[S_pos[0]][S_pos[1]] = 1

    while q:
        i, j = q.popleft()

        for di, dj in direcciones:
            ni, nj = i + di, j + dj
            if 0 <= ni < filas and 0 <= nj < cols and mapa[ni][nj] != 'H':
                if distancia[ni][nj] == -1:  # Primera visita
                    distancia[ni][nj] = distancia[i][j] + 1
                    contador[ni][nj] = contador[i][j]
                    q.append((ni, nj))
                elif distancia[ni][nj] == distancia[i][j] + 1:  # Ruta alternativa mínima
                    contador[ni][nj] += contador[i][j]

    pasos = distancia[G_pos[0]][G_pos[1]]
    num_caminos = contador[G_pos[0]][G_pos[1]] if pasos != -1 else 0

    return num_caminos, pasos

In [4]:
mapa_4x4 = [
    "S···",
    "·H·H",
    "···H",
    "H··G"
]

valido, mensaje = validar_mapa(mapa_4x4)
print(f"¿Mapa válido? {valido}, Mensaje: {mensaje}")

num_caminos, pasos = contar_caminos_minimos(mapa_4x4)
print(f"Caminos mínimos: {num_caminos}, Pasos requeridos: {pasos}")

¿Mapa válido? True, Mensaje: Mapa 4x4 válido
Caminos mínimos: 3, Pasos requeridos: 6


In [5]:
mapa = [
                "S·······",
                "·HHH·HH·",
                "·H·H····",
                "·H···H·H",
                "···H····",
                "·HH···H·",
                "·H··H·H·",
                "·······G"
            ]

valido, mensaje = validar_mapa(mapa)
print(f"¿Mapa válido? {valido}, Mensaje: {mensaje}")

num_caminos, pasos = contar_caminos_minimos(mapa)
print(f"Caminos mínimos: {num_caminos}, Pasos requeridos: {pasos}")

¿Mapa válido? True, Mensaje: Mapa 8x8 válido
Caminos mínimos: 5, Pasos requeridos: 14


## Función generadora de mapas
#### Parámetros
1. n: Dimensión del mapa (n x n)
2. num_caminos_min=1: Número mínimo de caminos mínimos requeridos
3. extremos=True: bandera: si True, S en (0,0) y G en (n-1, n-1); si False, posiciones aleatorias
4. densidad_hoyos=0.2: Probabilidad de generar un hoyo (0.0 a 0.5)
5. max_intentos=100: Intentos máximos para encontrar solución válida. Evita bucles sin fin.

#### Características principales

1. Memoización inteligente:
- Almacena soluciones válidas usando hash de parámetros
- Evita recalcular para mismas configuraciones
- lru_cache para optimizar acceso
2. Generación de caminos mínimos:
- Crea siempre al menos 1 camino mínimo garantizado
- Camino generado aleatoriamente (no siempre el mismo)
- Considera cualquier posición de S y G
3. Estrategia de reserva:
- Si falla después de max_intentos, genera mapa sin hoyos
- Garantiza siempre retornar un mapa válido
4. Validación integrada:
- Usa las funciones validar_mapa y contar_caminos_minimos
- Solo retorna mapas que cumplen los requisitos


**Nota**  
Los "Ejemplos de uso" se añaden en la misma celda del Notebook que la función ```generar_mapa``` para que cada vez que se pida un nuevo mapa se ejecute una nueva semilla de números aleatorios. Si separamos en dos celdas del Notebook la función y la ejecución de los ejemplos, la semilla no cambia y nos da siempre el mismo mapa.

In [34]:
import random
from collections import deque
import hashlib
from functools import lru_cache
#import time
#random.seed(int(time.time() * 1000) % (2**32))
random.seed()

# Memoización para almacenar patrones de caminos válidos
CAMINOS_MEMO = {}

def generar_mapa(n, num_caminos_min=1, extremos=True, densidad_hoyos=0.2, max_intentos=100):
    """
    Genera un mapa válido de Frozen Lake con las características especificadas.

    Args:
        n: Dimensión del mapa (n x n)
        num_caminos_min: Número mínimo de caminos mínimos requeridos
        extremos: Si True, S en (0,0) y G en (n-1, n-1). Si False, posiciones aleatorias
        densidad_hoyos: Probabilidad de generar un hoyo (0.0 a 0.5)
        max_intentos: Intentos máximos para encontrar solución válida

    Returns:
        Lista de strings representando el mapa
    """
    # Validación de parámetros
    if n < 2:
        raise ValueError("n debe ser al menos 2")
    if num_caminos_min < 1:
        raise ValueError("num_caminos_min debe ser al menos 1")
    if not (0.0 <= densidad_hoyos <= 0.5):
        raise ValueError("densidad_hoyos debe estar entre 0.0 y 0.5")

    # Generar clave única para memoización
    params_hash = hashlib.md5(f"{n}-{num_caminos_min}-{extremos}-{densidad_hoyos}".encode()).hexdigest()

    # Verificar solución memoizada
    if params_hash in CAMINOS_MEMO:
        return CAMINOS_MEMO[params_hash]

    # Caracteres del mapa
    char_hielo = '·'
    char_hoyo = 'H'

    for intento in range(max_intentos):
        # Crear mapa base lleno de hielo
        mapa = [[char_hielo] * n for _ in range(n)]

        # Posicionar S y G
        if extremos:
            S, G = (0, 0), (n-1, n-1)
        else:
            S = (random.randint(0, n-1), random.randint(0, n-1))
            G = S
            while G == S:
                G = (random.randint(0, n-1), random.randint(0, n-1))

        mapa[S[0]][S[1]] = 'S'
        mapa[G[0]][G[1]] = 'G'

        # Generar camino mínimo garantizado
        camino = generar_camino_minimo(S, G, n)

        # Marcar celdas del camino como seguras
        for i, j in camino:
            if (i, j) not in (S, G):
                mapa[i][j] = char_hielo

        # Rellenar el resto del mapa
        for i in range(n):
            for j in range(n):
                if mapa[i][j] == char_hielo and (i, j) not in camino:
                    if random.random() < densidad_hoyos:
                        mapa[i][j] = char_hoyo

        # Convertir a strings
        mapa_str = [''.join(fila) for fila in mapa]

        # Validar y contar caminos
        if not validar_mapa(mapa_str)[0]:
            continue

        num_caminos, _ = contar_caminos_minimos(mapa_str)

        if num_caminos >= num_caminos_min:
            # Almacenar en memo antes de retornar
            CAMINOS_MEMO[params_hash] = mapa_str
            return mapa_str

    # Intento de reserva si no se encontró solución
    return generar_mapa_reserva(n, extremos, num_caminos_min)

def generar_camino_minimo(S, G, n):
    """
    Genera un camino mínimo aleatorio entre S y G
    """
    # Calcular diferencias
    di = G[0] - S[0]
    dj = G[1] - S[1]

    # Generar secuencia de movimientos
    movimientos = []
    if di > 0: movimientos.extend(['abajo'] * di)
    else: movimientos.extend(['arriba'] * abs(di))
    if dj > 0: movimientos.extend(['derecha'] * dj)
    else: movimientos.extend(['izquierda'] * abs(dj))

    random.shuffle(movimientos)

    # Reconstruir camino
    camino = [S]
    i, j = S

    for mov in movimientos:
        if mov == 'arriba': i -= 1
        elif mov == 'abajo': i += 1
        elif mov == 'izquierda': j -= 1
        elif mov == 'derecha': j += 1
        camino.append((i, j))

    return camino

def generar_mapa_reserva(n, extremos, min_caminos):
    """Genera un mapa de reserva cuando falla la creación principal"""
    # Mapa completamente libre de hoyos
    mapa = [['·'] * n for _ in range(n)]

    if extremos:
        mapa[0][0] = 'S'
        mapa[n-1][n-1] = 'G'
    else:
        # Posiciones aleatorias no solapadas
        S = (random.randint(0, n-1), random.randint(0, n-1))
        G = S
        while G == S:
            G = (random.randint(0, n-1), random.randint(0, n-1))
        mapa[S[0]][S[1]] = 'S'
        mapa[G[0]][G[1]] = 'G'

    # Convertir a strings
    mapa_str = [''.join(fila) for fila in mapa]

    # Añadir a memo para futuras solicitudes iguales
    params_hash = hashlib.md5(f"{n}-{min_caminos}-{extremos}-0.0".encode()).hexdigest()
    CAMINOS_MEMO[params_hash] = mapa_str

    return mapa_str



###########################################
###########   EJEMPLOS DE USO   ###########
###########################################


# Generar mapa 4x4 con 2 caminos mínimos
mapa = generar_mapa(n=4, num_caminos_min=2, densidad_hoyos=0.5)

for fila in mapa:
    print(fila)

# Generar mapa 8x8 con S/G aleatorios
mapa = generar_mapa(n=8, num_caminos_min=3, extremos=False, densidad_hoyos=0.4)

print()
for fila in mapa:
    print(fila)

print()
num_caminos, pasos = contar_caminos_minimos(mapa)
print(f"Caminos mínimos: {num_caminos}, Pasos requeridos: {pasos}")

S···
H·HH
H··H
H··G

·GH·H·H·
··H·····
··H···HH
····H··H
H·······
·····HHH
··H·HHHH
S·H··H·H

Caminos mínimos: 3, Pasos requeridos: 8
